In [1]:
import os

In [2]:
pwd

'd:\\vscode\\nyc_building_energy_efficiency_ratings\\nyc_energy_rating\\notebooks'

In [3]:
os.chdir("../")

In [12]:
pwd

'd:\\vscode\\nyc_building_energy_efficiency_ratings\\nyc_energy_rating'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionsConfig:
    root_dir :Path
    source_URL:str
    local_data_file : Path
    unzip_dir:Path

In [14]:
from src.constant import *
from src.utils.common import read_yaml,create_directories

In [43]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionsConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionsConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [44]:
import os
import pandas as pd
import urllib.request as request
import requests
import zipfile
from src.logger import logger
from src.utils.common import get_size

In [49]:
class DataIngestion:
    def __init__(self, config: DataIngestionsConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        

     
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except zipfile.BadZipFile as e:
            print(f"BadZipFile Error:{e}")



    def read_data(self):

        df = pd.read_csv(self.config.local_data_file)
        print(df.head(5))
        print(df.shape)

In [50]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.read_data()
except Exception as e:
    raise e

[2024-03-09 18:39:58,136: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-09 18:39:58,143: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-09 18:39:58,145: INFO: common: created directory at: artifacts]
[2024-03-09 18:39:58,147: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-09 18:39:58,150: INFO: 3980016563: File already exists of size: ~ 4 KB]
BadZipFile Error:File is not a zip file
   10_Digit_BBL  Street_Number     Street_Name  DOF_Gross_Square_Footage  \
0    1007610020            325  WEST 37 STREET                     48843   
1    1007610022            313  WEST 37 STREET                     82354   
2    1007610028            307  WEST 37 STREET                    102824   
3    1007610033            545        8 AVENUE                    166874   
4    1007610037            555        8 AVENUE                    162260   

  Energy_Efficiency_Grade  Energy_Star_1-100_Score  
0                       A        